# 01 Import libraries

In [1]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd

In [3]:
import importlib

## Import our own modules

In [4]:
!rm -r Learning-the-Optimal-Solution-Path
!git clone https://github.com/Cumberkid/Learning-the-Optimal-Solution-Path.git

rm: cannot remove 'Learning-the-Optimal-Solution-Path': No such file or directory
Cloning into 'Learning-the-Optimal-Solution-Path'...
remote: Enumerating objects: 1744, done.
remote: Counting objects: 100% (714/714), done.
remote: Compressing objects: 100% (355/355), done.
remote: Total 1744 (delta 542), reused 479 (delta 359), pack-reused 1030 (from 1)
Receiving objects: 100% (1744/1744), 13.25 MiB | 12.69 MiB/s, done.
Resolving deltas: 100% (1179/1179), done.


(Using Colab)

In [5]:
import sys

In [6]:
# Add the parent directory to sys.path
sys.path.append('/content/Learning-the-Optimal-Solution-Path')

In [7]:
import lib
importlib.reload(lib)

<module 'lib' from '/content/Learning-the-Optimal-Solution-Path/lib/__init__.py'>

In [8]:
from lib.lsp.basis_generator import dampen_laguerre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import lsp_boosting
from lib.lsp.loss_fn_lsp import reg_exp_weighted_logit
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Load data

In [9]:
# file path for Colab. May need to change this
X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')

In [10]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [11]:
train_X = torch.tensor(X, dtype=torch.float32)
train_y = torch.tensor(y, dtype=torch.float32)

In [12]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=True, )
# stochastic gradient descent uses mini-batch
SGD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=20, shuffle=True, )
# test data
test_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=False, )

In [13]:
lam_max = 20
lam_min = 0
input_dim = X.shape[1]
loss_fn = reg_exp_weighted_logit

In [14]:
# Read the CSV file into a DataFrame
# truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/results/exact_soln_list.csv')
truth = pd.read_csv('exact_soln_list_laguerre.csv')

# Display the DataFrame
truth

,losses,theta_0,theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,...,theta_36,theta_37,theta_38,theta_39,theta_40,theta_41,theta_42,theta_43,theta_44,theta_45
0,0.251592,-0.290164,0.186074,0.232884,-0.027476,0.166136,0.110354,0.142358,0.009975,0.157656,...,0.000471,0.023097,-0.007038,0.003151,0.004510,-0.024440,0.000471,-0.129229,-0.096597,-0.041242
1,0.251645,-0.290092,0.186073,0.232887,-0.027478,0.166136,0.110326,0.142336,0.009961,0.157669,...,0.000487,0.023069,-0.007039,0.003154,0.004514,-0.024434,0.000487,-0.129220,-0.096579,-0.041224
2,0.251698,-0.290020,0.186071,0.232891,-0.027481,0.166136,0.110298,0.142315,0.009946,0.157682,...,0.000503,0.023042,-0.007041,0.003157,0.004518,-0.024429,0.000503,-0.129210,-0.096561,-0.041206
3,0.251751,-0.289947,0.186070,0.232895,-0.027484,0.166136,0.110270,0.142294,0.009931,0.157695,...,0.000519,0.023015,-0.007042,0.003160,0.004522,-0.024423,0.000519,-0.129201,-0.096543,-0.041188
4,0.251804,-0.289874,0.186068,0.232898,-0.027487,0.166136,0.110242,0.142272,0.009916,0.157708,...,0.000534,0.022988,-0.007043,0.003162,0.004526,-0.024417,0.000534,-0.129191,-0.096525,-0.041170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.425709,0.608973,0.015982,0.030750,-0.012680,0.033522,-0.189898,-0.060873,-0.075246,0.138251,...,0.104496,-0.190631,-0.003899,0.016846,0.022910,0.049139,0.104496,0.117941,0.139251,0.161150
1020,0.405809,0.627298,0.006731,0.017247,-0.013635,0.025534,-0.197376,-0.075369,-0.070611,0.125166,...,0.104706,-0.193085,-0.002800,0.016029,0.021890,0.052167,0.104706,0.126507,0.144123,0.163584
1021,0.382318,0.647696,-0.004087,0.001661,-0.014793,0.016128,-0.205935,-0.094402,-0.064959,0.109066,...,0.104903,-0.195830,-0.001589,0.015089,0.020769,0.055620,0.104903,0.136154,0.149574,0.166138
1022,0.354260,0.670989,-0.016946,-0.016535,-0.016181,0.004871,-0.216011,-0.119855,-0.058109,0.089182,...,0.105111,-0.199002,-0.000252,0.014026,0.019573,0.059569,0.105111,0.147241,0.155866,0.168880


In [15]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()

In [16]:
# import time

# 03 Exact gradient

In [17]:
max_epochs = 300

In [18]:
phi_lam = dampen_laguerre

In [19]:
start_basis_dim = 5
end_basis_dim = 13
init_lr = 0.0625

In [51]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/7 + 1.3)

In [53]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim,
                                        phi_lam, max_epochs, GD_data_loader, test_data_loader,
                                        loss_fn, lam_min, lam_max, true_losses, init_lr=init_lr,
                                        q=1.5, thresh_basis=thresh_basis, record_frequency=10,
                                        distribution='exponential', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 0.17033571004867554
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.12579107284545898
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.1772816181182862
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.290246456861496
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.29006937146186834
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.24645903706550604
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.19762173295021063
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.10872614383697515
--------approximate solution path for # itr = 90 complete-

In [54]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_exact_boosted_breaks_laguerre.csv', index=False)

In [55]:
file_path = 'LSP_results_exact_boosted_laguerre.csv'

LSP_results_exact = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,0.170336
1,20.0,0.125791
2,30.0,0.177282
3,40.0,0.290246
4,50.0,0.290069
...,...,...
73,740.0,0.003244
74,750.0,0.002977
75,760.0,0.002820
76,770.0,0.002655


# Noisy gradient oracle

In [ ]:
max_epochs = 200

In [ ]:
phi_lam = scaled_shifted_legendre

In [ ]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/5 + 5) * (50**2)

In [ ]:
# beta/sqrt(t)
def step_size(t, beta):
    return beta/np.sqrt(t+1)

In [ ]:
start_basis_dim = 5
end_basis_dim = 10
beta = 0.5

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim, phi_lam, max_epochs,
                                                        SGD_data_loader, test_data_loader, loss_fn, lam_min, lam_max,
                                                        true_losses, step_size=step_size, const=beta, weighted_avg=True,
                                                        avg_upon_prev=True, thresh_basis=thresh_basis, record_frequency=10,
                                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 0.11422920227050781
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.01976636052131653
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.009443342685699463
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.002981826663017273
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.002051115036010742
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.0017272531986236572
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.0030344128608703613
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.0031889528036117554
--------approximate solution path for # itr = 

In [ ]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_noisy_boosted_breaks_legendre.csv', index=False)

In [ ]:
file_path = 'LSP_results_noisy_boosted_legendre.csv'

LSP_results_noisy = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_noisy.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,0.114229
1,20.0,0.019766
2,30.0,0.009443
3,40.0,0.002982
4,50.0,0.002051
...,...,...
88,890.0,0.000264
89,900.0,0.000271
90,910.0,0.000286
91,920.0,0.000291
